In [1]:
!pip install torch pandas numpy matplotlib seaborn wordcloud datasets transformers scikit-learn requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import requests
import json
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


In [5]:


from datasets import load_dataset, DatasetDict




# Function to fetch data from Hugging Face API
def fetch_data(offset=0, length=500):
    url = f"https://datasets-server.huggingface.co/rows?dataset=abisee%2Fcnn_dailymail&config=1.0.0&split=train&offset={offset}&length={length}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()["rows"]
        articles = [row["row"]["article"] for row in data]
        highlights = [row["row"]["highlights"] for row in data]
        return articles, highlights
    else:
        print("Error fetching data:", response.status_code)
        return [], []

# Fetching 2000 samples
articles, summaries = fetch_data(offset=0, length=2000)

# Convert to Hugging Face dataset format
dataset = DatasetDict({
    "train": {"article": articles[:1600], "summary": summaries[:1600]},
    "test": {"article": articles[1600:], "summary": summaries[1600:]}
})

print("Dataset successfully loaded!")
print(f"Train Samples: {len(dataset['train']['article'])}, Test Samples: {len(dataset['test']['article'])}")


Error fetching data: 422
Dataset successfully loaded!
Train Samples: 0, Test Samples: 0


In [6]:
# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model loaded and using device: {device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded and using device: cuda


In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Define a preprocessing function for tokenization
def preprocess_function(examples):
    inputs = ["summarize: " + article for article in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply tokenization using .map()
tokenized_dataset = dataset.map(preprocess_function, batched=True)

print("Data tokenized successfully!")


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

✅ Data tokenized successfully!


In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load only a subset of the dataset (e.g., 10,000 training samples, 2,000 test samples)
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Reduce dataset size
small_train = dataset["train"].shuffle(seed=42).select(range(10000))
small_test = dataset["test"].shuffle(seed=42).select(range(2000))

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + article for article in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_test = small_test.map(preprocess_function, batched=True)

print("Reduced dataset tokenized successfully!")


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Reduced dataset tokenized successfully!


In [10]:
import wandb

wandb.login(key="5def3487c6c793856b760687a6e481be35b8d804")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmeroshaan290 (ahmeroshaan290-fast-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
import matplotlib.pyplot as plt
from transformers import TrainerCallback

# Custom Callback to store loss values
class LossHistoryCallback(TrainerCallback):
    def __init__(self):
        self.losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            self.losses.append(logs["loss"])

# Initialize loss tracker
loss_callback = LossHistoryCallback()


In [ ]:
from transformers import TrainingArguments, Trainer

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-4,
    weight_decay=0.01,
    num_train_epochs=20,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    callbacks=[loss_callback],  # Attach loss callback
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
def summarize_text(text, model, tokenizer):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)

    # Generate summary
    summary_ids = model.generate(**inputs, max_length=150, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Format output beautifully
    print("\n" + "=" * 80)
    print("INPUT TEXT:\n" + text)
    print("-" * 80)
    print("SUMMARY:\n" + " " * 5 + generated_summary + " " * 5)
    print("=" * 80 + "\n")

# Take user input
user_text = input("Enter a long article to summarize:\n")
user_text="""Artificial Intelligence (AI) is revolutionizing various aspects of human life, including business, healthcare, education, and entertainment. AI-powered systems are now capable of performing tasks that were previously limited to human intelligence, such as speech recognition, language translation, decision-making, and even creative writing. Companies like Google, Microsoft, and OpenAI have developed advanced AI models that can analyze vast amounts of data, identify patterns, and generate insights at an unprecedented speed.

In the healthcare industry, AI has enabled early disease detection, personalized treatment plans, and robotic-assisted surgeries. AI-driven algorithms analyze medical images, helping doctors diagnose conditions like cancer and neurological disorders with higher accuracy. Additionally, wearable health devices powered by AI monitor patient vitals and provide real-time health recommendations.

The education sector has also benefited significantly from AI. AI-powered learning platforms personalize educational content for students, adapting to their learning pace and style. Virtual tutors and AI chatbots assist students in answering queries, making learning more interactive and accessible. However, concerns remain regarding the over-reliance on AI in education, as human interaction plays a crucial role in cognitive development.

While AI has led to significant advancements, it has also raised ethical and societal concerns. The automation of jobs threatens employment in various industries, leading to discussions about universal basic income and reskilling workers. Bias in AI models is another pressing issue, as AI systems trained on biased data can lead to discriminatory outcomes. Additionally, the rise of deepfake technology has sparked debates about misinformation and digital security.

Despite these challenges, AI continues to evolve rapidly. Researchers and policymakers are working on establishing regulations to ensure the ethical use of AI. Transparency in AI decision-making, data privacy protection, and responsible AI development are key areas of focus. As AI becomes more integrated into everyday life, balancing innovation with ethical considerations will be crucial for its sustainable growth."""


summarize_text(user_text, model, tokenizer)
